In [1]:
# 패키지 임포트
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle5 as pickle
sns.set()

In [2]:
# 제작한 모듈들 임포트
from price.price_processing import *

from backtest.metric import Metric
from strategy.factors.momentum import *
from strategy.optimize.cross_sectional import *
from strategy.optimize.time_series import *

In [3]:
with open('../../part07.-퀀트-트레이딩-실무-실습-파일/chapter3/피클파일/equity_price.pkl', 'rb') as f:
    equity_price = pickle.load(f)
    equity_price.index = pd.to_datetime(equity_price.index)
    equity_price.index.name = 'date_time'
    
equity_price = equity_price.drop(columns='DOW')

In [4]:
target_rebal = rebal_dates(price=equity_price, period='month')

In [5]:
rebal_price_df = price_on_rebal(equity_price, target_rebal)
rebal_price_df

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,GS,...,MRK,MSFT,NKE,PG,TRV,UNH,V,VZ,WBA,WMT
2010-01-29,6.859,58.480,37.66,60.60,52.24,15.8875,22.47,72.12,29.55,148.72,...,36.4056,28.1800,15.9375,61.55,50.67,33.00,20.5075,27.4898,36.05,53.43
2010-02-26,7.308,56.610,38.19,63.16,57.05,16.9875,24.33,72.30,31.24,156.35,...,35.1660,28.6700,16.9000,63.28,52.59,33.86,21.3200,27.0319,35.24,54.07
2010-03-31,8.393,59.838,41.26,72.61,62.85,18.6125,26.03,75.83,34.91,170.63,...,35.6142,29.2875,18.3750,63.27,53.94,32.67,22.7575,28.9848,37.09,55.60
2010-04-30,9.325,57.310,46.12,72.43,68.09,21.4000,26.93,81.44,36.84,145.20,...,33.4116,30.5350,18.9775,62.16,50.74,30.31,22.5575,27.0039,35.15,53.64
2010-05-28,9.174,51.780,39.87,64.18,60.76,21.6325,23.16,73.87,33.42,144.26,...,32.1243,25.8000,18.0950,61.09,49.47,29.07,18.1150,25.7144,32.04,50.56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-30,136.720,243.300,138.62,136.72,178.76,165.0400,42.64,144.78,94.40,297.02,...,91.1700,256.8300,102.2000,143.79,169.13,513.63,196.8900,50.7500,37.90,121.58
2022-07-29,162.510,247.470,154.02,159.31,198.25,184.0200,45.37,163.78,106.10,333.39,...,89.3400,280.7400,114.9200,138.91,158.70,542.34,212.1100,46.1900,39.62,132.05
2022-08-31,157.220,240.300,152.00,160.25,184.71,156.1200,44.72,158.06,112.08,332.67,...,85.3600,261.4700,106.4500,137.94,161.64,519.33,198.7100,41.8100,35.06,132.55
2022-09-30,138.200,225.400,134.91,121.08,164.08,143.8400,40.00,143.67,94.33,293.05,...,86.1200,232.9000,83.1200,126.25,153.20,505.04,177.6500,37.9700,31.40,129.70


### momentum long-short test

In [6]:
momentum = MomentumFactor(rebal_price_df, 6, 5, long_only=True)
signal = momentum.dual_momentum()

In [7]:
signal

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,GS,...,MRK,MSFT,NKE,PG,TRV,UNH,V,VZ,WBA,WMT
2010-07-30,1,0,1,0,1,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2010-08-31,1,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2010-09-30,1,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2010-10-29,1,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
2010-11-30,0,0,0,0,1,1,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-30,0,1,0,0,0,0,0,1,0,0,...,1,0,0,0,1,0,0,0,0,0
2022-07-29,0,1,0,0,0,0,0,1,0,0,...,1,0,0,0,0,1,0,0,0,0
2022-08-31,0,1,0,0,0,0,0,1,0,0,...,1,0,0,0,0,1,0,0,0,0
2022-09-30,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


### cross sectional test

In [8]:
cs_weight = Equalizer(signal=signal, rebal_price=rebal_price_df, param=12)
beta_weight = cs_weight.beta()
ew_weight = cs_weight.ew()
emv_weight = cs_weight.emv()

msr_weight = Optimization(signal=signal, rebal_price=rebal_price_df, param=12, call_method='msr').run()
gmv_weight = Optimization(signal=signal, rebal_price=rebal_price_df, param=12, call_method='gmv').run()
mdp_weight = Optimization(signal=signal, rebal_price=rebal_price_df, param=12, call_method='mdp').run()
rp_weight = Optimization(signal=signal, rebal_price=rebal_price_df, param=12, call_method='rp').run()


In [9]:
cs_model = {'BETA': beta_weight,
            'EW': ew_weight, 
            'EMV': emv_weight, 
            'MSR': msr_weight, 
            'GMV': gmv_weight, 
            'MDP': mdp_weight, 
            'RP': rp_weight
            }

In [10]:
beta_weight

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,GS,...,MRK,MSFT,NKE,PG,TRV,UNH,V,VZ,WBA,WMT
2010-07-30,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,...,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483
2010-08-31,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,...,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483
2010-09-30,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,...,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483
2010-10-29,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,...,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483
2010-11-30,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,...,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-06-30,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,...,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483
2022-07-29,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,...,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483
2022-08-31,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,...,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483
2022-09-30,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,...,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483,0.034483


### backtest of (momentum + cross sectional)

In [11]:
for name, weight in cs_model.items():
    print(name)
    cum_rets = port_cum_rets(calculate_portvals(price_df=equity_price, weight_df=weight, signal_df=signal, long_only=True))
    #display(Metric(cum_rets).rolling_metric()['dd'].plot())
    #plt.show()
    Metric(portfolio=cum_rets).print_report()
    print()

BETA
Annualized Return: 11.74%
Annualized Volatility: 16.85%
Skewness: -0.41
Kurtosis: 15.79
Max Drawdown: -33.08%
Max Drawdown Duration: 330 days
Annualized Sharp Ratio: 0.46
Annualized Sortino Ratio: -0.16
Annualized Calmar Ratio: 0.35
Annualized VaR: -0.03
Annualized VaR Ratio: 0.02
Annualized CVaR: -0.04
Annualized CVaR Ratio: 0.01
Annualized hit Ratio: 0.55
Annualized GtP Ratio: 0.94

EW
Annualized Return: 14.44%
Annualized Volatility: 19.62%
Skewness: -0.52
Kurtosis: 16.73
Max Drawdown: -28.59%
Max Drawdown Duration: 500 days
Annualized Sharp Ratio: 0.53
Annualized Sortino Ratio: -0.10
Annualized Calmar Ratio: 0.51
Annualized VaR: -0.03
Annualized VaR Ratio: 0.02
Annualized CVaR: -0.05
Annualized CVaR Ratio: 0.01
Annualized hit Ratio: 0.55
Annualized GtP Ratio: 0.96

EMV
Annualized Return: 15.85%
Annualized Volatility: 20.29%
Skewness: -0.50
Kurtosis: 13.67
Max Drawdown: -28.77%
Max Drawdown Duration: 617 days
Annualized Sharp Ratio: 0.58
Annualized Sortino Ratio: -0.08
Annualize

### backtest of (momentum + cross sectional + time series)

In [12]:
# TimeSeries의 변수로 들어갈 포트폴리오 리턴을 만든다. 
def port_val(weight_df, rebal_price): 
    
    # 포트폴리오 수익률 데이터프레임
    rets = rebal_price.pct_change().fillna(0)
    port_rets = (weight_df * rets).dropna()
    port_rets = port_rets.sum(axis=1)
    
    return port_rets

port_rets = port_val(weight_df=ew_weight, rebal_price=rebal_price_df)
port_rets

2010-07-30    0.110310
2010-08-31    0.010702
2010-09-30    0.103135
2010-10-29    0.024252
2010-11-30    0.060904
                ...   
2022-06-30   -0.059107
2022-07-29    0.040845
2022-08-31   -0.033753
2022-09-30    0.008903
2022-10-17    0.083554
Length: 148, dtype: float64

In [13]:
# 종적 자산배분 
ew_weights, ew_ew_weight = TimeSeries(port_rets=port_rets, cs_weight=ew_weight, call_method='ew', param=12).run()
vt_weights, ew_vt_weight = TimeSeries(port_rets=port_rets, cs_weight=ew_weight, call_method='vt', param=12).run()
cvt_weights, ew_cvt_weight = TimeSeries(port_rets=port_rets, cs_weight=ew_weight, call_method='cvt', param=12).run()
kl_weights, ew_kl_weight = TimeSeries(port_rets=port_rets, cs_weight=ew_weight, call_method='kl', param=12).run()
cppi_weights, ew_cppi_weight = TimeSeries(port_rets=port_rets, cs_weight=ew_weight, call_method='cppi', param=12).run()

In [14]:
ts_models = {'ew': ew_ew_weight,
            'vt': ew_vt_weight, 
            'cvt': ew_cvt_weight, 
            'kl': ew_kl_weight, 
            'cppi': ew_cppi_weight
            }

In [15]:
# 가격 df에 현금 추가
equity_price = add_cash(equity_price, 252, 0.04)

In [18]:
for name, weight in ts_models.items():
    print(name)
    cum_rets = port_cum_rets(calculate_portvals(price_df=equity_price, weight_df=weight, signal_df=signal, long_only=True))
    #display(Metric(cum_rets).rolling_metric()['dd'].plot())
    #plt.show()
    Metric(portfolio=cum_rets).print_report()
    print()

ew
Annualized Return: 11.51%
Annualized Volatility: 13.65%
Skewness: -0.60
Kurtosis: 16.04
Max Drawdown: -20.56%
Max Drawdown Duration: 500 days
Annualized Sharp Ratio: 0.55
Annualized Sortino Ratio: -0.23
Annualized Calmar Ratio: 0.56
Annualized VaR: -0.02
Annualized VaR Ratio: 0.02
Annualized CVaR: -0.03
Annualized CVaR Ratio: 0.01
Annualized hit Ratio: 0.55
Annualized GtP Ratio: 0.95

vt
Annualized Return: 9.66%
Annualized Volatility: 15.26%
Skewness: -0.89
Kurtosis: 44.07
Max Drawdown: -28.59%
Max Drawdown Duration: 472 days
Annualized Sharp Ratio: 0.37
Annualized Sortino Ratio: -0.20
Annualized Calmar Ratio: 0.34
Annualized VaR: -0.03
Annualized VaR Ratio: 0.02
Annualized CVaR: -0.04
Annualized CVaR Ratio: 0.01
Annualized hit Ratio: 0.59
Annualized GtP Ratio: 0.80

cvt
Annualized Return: 15.55%
Annualized Volatility: 76.37%
Skewness: 15.09
Kurtosis: 505.89
Max Drawdown: -90.97%
Max Drawdown Duration: 500 days
Annualized Sharp Ratio: 0.15
Annualized Sortino Ratio: 0.07
Annualized C